In [ ]:
import eikon as ek
import refinitiv.data as rd
import pandas as pd
import os
import sys
import datetime as dt
from textblob import TextBlob

from retrying import retry
from dotenv import load_dotenv
from tqdm import tqdm
from bs4 import BeautifulSoup

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src_HF')
from utils.main_utils import *

load_dotenv()
ek.set_app_key(os.getenv('EIKON_API_KEY'))
rd.open_session(app_key=os.getenv('EIKON_API_KEY'))

In [ ]:
headline_topic = 'CRU'

headline_df = pd.read_csv(rf'C:\Users\joneh\master_thesis\data\raw_news_headlines\EIKON_{headline_topic}_NEWS.csv')

# check if file exists
if not os.path.exists(rf'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.csv'):
    blank_df = pd.DataFrame(columns=['date', 'versionCreated', 'text', 'storyId', 'sourceCode', 'fullStory'])
    blank_df.to_csv(rf'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.csv', index=False)

existing_df = pd.read_csv(rf'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.csv')
existing_ids = existing_df['storyId']

# get all story ids not in the existing_ids series
storie_ids = headline_df[~headline_df['storyId'].isin(existing_ids)]['storyId']

print(f'Number of stories alredy downloaded/failed: ({len(existing_ids)}/{len(headline_df)})')

text_dict = dict()

for id in tqdm(storie_ids):
    try:
        response = ek.get_news_story(id)
        soup = BeautifulSoup(response, 'html.parser')
        text = soup.get_text()
        text_dict[id] = text
    except ek.EikonError as e:
        if e == 'Error code 429 | Client Error: Too many requests, please try again later.':
            break
        else:
            print('Error')
            text_dict[id] = 'error'


In [ ]:
text_df = pd.DataFrame.from_dict(text_dict, orient='index', columns=['text'])

display(text_df)

# map the text to the headlines
headline_df['fullStory'] = headline_df['storyId'].map(text_df['text'])
complete_df = pd.concat([existing_df, headline_df.dropna(subset=['fullStory'])])

complete_df.to_csv(rf'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_{headline_topic}_NEWS_FULL.csv', index=False)

In [ ]:
pd.read_csv(rf'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_CRU_NEWS_FULL.csv')